In [1]:
import numpy as np 
import pandas as pd 
from glob import glob
from scipy import stats
from sklearn.metrics import confusion_matrix, classification_report
from scipy.stats import ttest_ind


In [61]:
inceptionResNetV2_cm=np.array([[36,16],[55,127]])
efficientNetB3_cm=np.array([[37,15],[37,145]])
efficientNetB2_cm=np.array([[43,9],[43,139]])
y_test=np.ones([52])
y_test=np.concatenate([y_test,np.zeros([182])])

inceptionResNetV2_pred=np.zeros([len(y_test)])
numbers = np.random.choice(range(0, 52), 36, replace=False)
inceptionResNetV2_pred[numbers]=1
numbers = np.random.choice(range(52, len(y_test)), 55, replace=False)
inceptionResNetV2_pred[numbers]=1
efficientNetB2_pred=np.copy(inceptionResNetV2_pred)
ttp=np.where(efficientNetB2_pred[:52]==0)[0]
numbers = np.random.choice(range(0, len(ttp)), 1, replace=False)
efficientNetB2_pred[ttp[numbers]]=1
ffp=np.where(efficientNetB2_pred[52:]==1)[0]+52
numbers = np.random.choice(range(0, len(ffp)), 55-37, replace=False)
efficientNetB2_pred[ffp[numbers]]=0
efficientNetB3_pred=np.copy(inceptionResNetV2_pred)
ttp=np.where(efficientNetB3_pred[:52]==0)[0]
numbers = np.random.choice(range(0, len(ttp)), 43-36, replace=False)
efficientNetB3_pred[ttp[numbers]]=1
ffp=np.where(efficientNetB3_pred[52:]==1)[0]+52
numbers = np.random.choice(range(0, len(ffp)), 55-43, replace=False)
efficientNetB3_pred[ffp[numbers]]=0

In [93]:
import numpy as np
import pandas as pd
from scipy.stats import t, ttest_ind, f_oneway
from statsmodels.stats.multitest import multipletests
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# 가상의 예제 데이터 생성
data_model1 = inceptionResNetV2_pred
data_model2 = efficientNetB2_pred
data_model3 = efficientNetB3_pred

# 이진분류 모델 3개의 결과를 각각 저장
results_model1 = data_model1 > 0
results_model2 = data_model2 > 0
results_model3 = data_model3 > 0

true_labels = y_test

# 성능 지표 계산 함수
def calculate_performance_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    f1 = f1_score(y_true, y_pred)
    return accuracy, sensitivity, specificity, f1

alpha_values = [0.2, 0.15, 0.1, 0.05, 0.025, 0.01, 0.005, 0.001]

# 결과를 저장할 DataFrame 생성
result_df = pd.DataFrame(columns=['Model', 'Metric', 'Value','Alpha', 'Confidence Interval', 'p-value', 'ANOVA with bonferroni collection p-value'])

# 각 모델에 대한 신뢰구간, p-value, ANOVA p-value 계산 및 DataFrame에 저장
for model, results in zip(['inceptionResNetV2', 'efficientNetB2', 'efficientNetB3'], [results_model1, results_model2, results_model3]):
    accuracy, sensitivity, specificity, f1 = calculate_performance_metrics(true_labels, results)

    # Calculate p-value using t-test
    t_statistic, p_value_ttest = ttest_ind(results, true_labels)

    # Perform ANOVA
    _, p_value_anova = f_oneway(results_model1, results_model2, results_model3)

    # Apply Bonferroni correction to ANOVA p-value
    _, p_value_anova_corrected, _, _ = multipletests([p_value_anova], method='bonferroni')

    for metric_name, metric_value in zip(['Accuracy', 'Sensitivity', 'Specificity', 'F1 Score'], [accuracy, sensitivity, specificity, f1]):
        for alpha in alpha_values:
            # Calculate standard error of the mean
            sem = np.std(results) / np.sqrt(len(results))

            # Adjust margin_of_error and confidence_interval based on the metric
            if metric_name == 'Accuracy':
                margin_of_error = t.ppf(1 - alpha / 2, len(results) - 1) * sem
            else:
                z_score = norm.ppf(1 - alpha / 2)
                margin_of_error = z_score * np.sqrt((metric_value * (1 - metric_value)) / len(results))

            confidence_interval = (metric_value - margin_of_error, metric_value + margin_of_error)

            # 결과를 DataFrame에 추가
            result_df = pd.concat([result_df, pd.DataFrame({
                'Model': [model],
                'Metric': [metric_name],
                'Value': [metric_value],
                'Alpha': [(1 - alpha) * 100],
                'Confidence Interval': [(confidence_interval[1] - confidence_interval[0])/2],
                'p-value': [p_value_ttest],
                'ANOVA with bonferroni collection p-value': [p_value_anova_corrected[0]]
            })])

# 결과 DataFrame 출력
print(result_df)
# 결과 DataFrame을 CSV 파일로 저장 (필요에 따라 주석 처리)
result_df.to_csv('model_comparison_results.csv', index=False)


                Model    Metric     Value  Alpha  Confidence Interval  \
0   inceptionResNetV2  Accuracy  0.696581   80.0             0.081915   
0   inceptionResNetV2  Accuracy  0.696581   85.0             0.092056   
0   inceptionResNetV2  Accuracy  0.696581   90.0             0.105257   
0   inceptionResNetV2  Accuracy  0.696581   95.0             0.125575   
0   inceptionResNetV2  Accuracy  0.696581   97.5             0.143791   
..                ...       ...       ...    ...                  ...   
0      efficientNetB3  F1 Score  0.623188   95.0             0.124177   
0      efficientNetB3  F1 Score  0.623188   97.5             0.142008   
0      efficientNetB3  F1 Score  0.623188   99.0             0.163197   
0      efficientNetB3  F1 Score  0.623188   99.5             0.177845   
0      efficientNetB3  F1 Score  0.623188   99.9             0.208478   

     p-value  ANOVA with bonferroni collection p-value  
0   0.000083                                  0.242462  
0   0.000

/tmp/ipykernel_3012593/1117742249.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame({


In [90]:
locals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'import numpy as np \nimport pandas as pd \nfrom glob import glob\nfrom scipy import stats\nfrom sklearn.metrics import confusion_matrix, classification_report\nfrom scipy.stats import ttest_ind',
  'inceptionResNetV2_cm=np.array([[36,16],[55,127]])\nefficientNetB2_cm=np.array([[43,9],[43,139]])\nefficientNetB3_cm=np.array([[37,15],[37,145]])\ny_test=np.ones([52])\ny_test=np.concatenate([y_test,np.zeros([182])])\ninceptionResNetV2_pred=np.zeros([len(y_test)])\ninceptionResNetV2_pred[:inceptionResNetV2_cm[0,0]]=1\ninceptionResNetV2_pred[52:52+inceptionResNetV2_cm[1,0]]=1\nefficientNetB2_pred=np.zeros([len(y_test)])\nefficientNetB2_pred[:efficientNetB2_cm[0,0]]=1\nefficientNetB2_pred[52:52+efficientNetB2_cm[1,0]]=

In [43]:
import pandas as pd

# CSV 파일에서 데이터 읽기
csv_file = 'model_comparison_results.csv'
df = pd.read_csv(csv_file)

# 데이터를 원하는 형식으로 가공
table = df.pivot_table(index=['Model', 'Metric'], columns='Alpha', values=['Value', 'Confidence Interval', 'p-value', 'ANOVA with bonferroni collection p-value'])

# 워드 논문 형식에 맞게 컬럼 정렬
table = table[[('Value', 0.2),
               ('Confidence Interval', 0.2), ('Confidence Interval', 0.15), ('Confidence Interval', 0.1), ('Confidence Interval', 0.05), ('Confidence Interval', 0.025), ('Confidence Interval', 0.01), ('Confidence Interval', 0.005), ('Confidence Interval', 0.001),
               ('p-value', 0.2),
               ('ANOVA with bonferroni collection p-value', 0.2), ('ANOVA with bonferroni collection p-value', 0.15), ('ANOVA with bonferroni collection p-value', 0.1), ('ANOVA with bonferroni collection p-value', 0.05), ('ANOVA with bonferroni collection p-value', 0.025), ('ANOVA with bonferroni collection p-value', 0.01), ('ANOVA with bonferroni collection p-value', 0.005), ('ANOVA with bonferroni collection p-value', 0.001)]]

# 워드 표 형식으로 출력
print(table.to_markdown())



|                                      |   ('Value', 0.2) |   ('Confidence Interval', 0.2) |   ('Confidence Interval', 0.15) |   ('Confidence Interval', 0.1) |   ('Confidence Interval', 0.05) |   ('Confidence Interval', 0.025) |   ('Confidence Interval', 0.01) |   ('Confidence Interval', 0.005) |   ('Confidence Interval', 0.001) |   ('p-value', 0.2) |   ('ANOVA with bonferroni collection p-value', 0.2) |   ('ANOVA with bonferroni collection p-value', 0.15) |   ('ANOVA with bonferroni collection p-value', 0.1) |   ('ANOVA with bonferroni collection p-value', 0.05) |   ('ANOVA with bonferroni collection p-value', 0.025) |   ('ANOVA with bonferroni collection p-value', 0.01) |   ('ANOVA with bonferroni collection p-value', 0.005) |   ('ANOVA with bonferroni collection p-value', 0.001) |
|:-------------------------------------|-----------------:|-------------------------------:|--------------------------------:|-------------------------------:|--------------------------------:|------------

In [27]:
result_df.loc[0, 'Metric']

0    Accuracy
0    Accuracy
0    Accuracy
0    Accuracy
0    Accuracy
       ...   
0         AUC
0         AUC
0         AUC
0         AUC
0         AUC
Name: Metric, Length: 120, dtype: object

In [78]:
import numpy as np
from scipy.stats import t, ttest_ind
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 가상의 예제 데이터 생성
np.random.seed(42)
data_model1 = np.random.normal(loc=0, scale=1, size=100)
data_model2 = np.random.normal(loc=0.5, scale=1, size=100)
data_model3 = np.random.normal(loc=1, scale=1, size=100)

# 이진분류 모델 3개의 결과를 각각 저장
results_model1 = (data_model1 > 0).astype(int)
results_model2 = (data_model2 > 0).astype(int)
results_model3 = (data_model3 > 0).astype(int)

# 실제 라벨 (예시로 모두 1로 설정)
true_labels = np.ones_like(data_model1)

# 성능 지표 계산 함수
def calculate_performance_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return accuracy, precision, recall, f1

# 각 모델에 대한 t-test 및 p-value 계산 및 출력
for model, results in zip(['Model 1', 'Model 2', 'Model 3'], [results_model1, results_model2, results_model3]):
    accuracy, precision, recall, f1 = calculate_performance_metrics(true_labels, results)
    
    print(f"\nT-test and p-value for {model}:")
    for metric_name, metric_value in zip(['Accuracy', 'Precision', 'Recall', 'F1 Score'], [accuracy, precision, recall, f1]):
        print(f"  {metric_name}: {metric_value:.2%}")
        t_statistic, p_value = ttest_ind(results, true_labels)
        print(f"    t-statistic: {t_statistic}, p-value: {p_value}")



T-test and p-value for Model 1:
  Accuracy: 46.00%
    t-statistic: -10.780417028313323, p-value: 1.2866965296964952e-21
  Precision: 100.00%
    t-statistic: -10.780417028313323, p-value: 1.2866965296964952e-21
  Recall: 46.00%
    t-statistic: -10.780417028313323, p-value: 1.2866965296964952e-21
  F1 Score: 63.01%
    t-statistic: -10.780417028313323, p-value: 1.2866965296964952e-21

T-test and p-value for Model 2:
  Accuracy: 68.00%
    t-statistic: -6.825557507934252, p-value: 1.0449223161892874e-10
  Precision: 100.00%
    t-statistic: -6.825557507934252, p-value: 1.0449223161892874e-10
  Recall: 68.00%
    t-statistic: -6.825557507934252, p-value: 1.0449223161892874e-10
  F1 Score: 80.95%
    t-statistic: -6.825557507934252, p-value: 1.0449223161892874e-10

T-test and p-value for Model 3:
  Accuracy: 85.00%
    t-statistic: -4.179783276115416, p-value: 4.377094840750872e-05
  Precision: 100.00%
    t-statistic: -4.179783276115416, p-value: 4.377094840750872e-05
  Recall: 85.00%


/home/gil/anaconda3/envs/LeeYS/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/home/gil/anaconda3/envs/LeeYS/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/home/gil/anaconda3/envs/LeeYS/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
